In [0]:
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

coau_df = pd.read_csv("data/coauthorsCC-edges.csv",delimiter=";")

G = nx.from_pandas_edgelist(coau_df,"Source","Target",['Weight'],create_using=nx.Graph())
nodos = list(G.nodes)

In [217]:
#Obteniendo dataframe de pares de nodos que no tienen coneccion

adj_G = nx.to_numpy_matrix(G, nodelist = nodos)
all_unconnected_pairs = []

offset = 0
for i in range(adj_G.shape[0]):
  for j in range(offset,adj_G.shape[1]):
    if i != j:
        #Para evitar tener demasiados ejemplos de pares de nodos que no tienen coneccion
        #Solo se tomaran solo los ejemplos de los pares de nodos cuya distancia mas corta
        #sea 2, y que no tengan coneccion
        if nx.shortest_path_length(G, str(nodos[i]), str(nodos[j])) ==2:
          if adj_G[i,j] == 0:
            all_unconnected_pairs.append((nodos[i],nodos[j]))

  offset = offset + 1

node_1_unlinked = [i[0] for i in all_unconnected_pairs]
node_2_unlinked = [i[1] for i in all_unconnected_pairs]

#Creando dataframe para almacenar a los pares de nodos no conectados que se recogieron
data = pd.DataFrame({'Source':node_1_unlinked,'Target':node_2_unlinked})
data['Weight'] = 0
data['link'] = 0

indices = range(len(data['Source']))

#Separando el dataframe en datos para train y para test
#(se busca un numero no muy grande de datos de pares que no tienen coneccion en los datos para entrenar, para
#evitar que los datos de entrenamiento esten desbalanceados)

removeIn, noRemoveIn = train_test_split(indices,test_size=0.25,random_state=32)

test= data.copy()
test = test.drop(index=noRemoveIn)

data = data.drop(index=removeIn)

data.shape #Cantidad de ejemplos negativos 

(6348, 4)
(2117, 4)


In [219]:
#Trabajando con los pares de nodos que si estan conectados

#Obteniendo lista de los links que se pueden borrar (que no eliminan nodos, o dividen al grafo en mas de una componente)

initial_node_count = len(G.nodes)

coau_df_temp = coau_df.copy()

omissible_links_index = []

for i in coau_df.index.values:
    # Eliminar una arista y construir un nuevo grafo sin esa unica arista
    G_temp = nx.from_pandas_edgelist(coau_df_temp.drop(index=i), "Source", "Target",["Weight"], create_using=nx.Graph())

    # Verificando que este par no parte el grafo, y que el numero de nodos siga siendo el mismo

    if (nx.number_connected_components(G_temp) == 1) and (len(G_temp.nodes) == initial_node_count):
        omissible_links_index.append(i)
        coau_df_temp = coau_df_temp.drop(i)


#creando dataframe de edges que se pueden remover

coau_df_temp2 = coau_df.copy()
coau_df_temp2["link"]=1

#Separando dataframe en datos para train y test
linkNoDele,linkDele = train_test_split(omissible_links_index,test_size=0.25,random_state=35)

testlinks = coau_df_temp2.loc[linkDele]
coau_df_temp = coau_df_temp2.drop(index=linkDele)

data = data.append(coau_df_temp[['Source', 'Target', 'link','Weight']], ignore_index=True)
data["Weight"] = data["Weight"].astype('int64')

test = test.append(testlinks[['Source', 'Target', 'link','Weight']], ignore_index=True)
data.shape

(4413, 4)

In [0]:
#Creando nuevo grafo con links eliminados aleatoriamente
G_data = nx.from_pandas_edgelist(coau_df_temp, "Source", "Target",["Weight"], create_using=nx.Graph())

In [0]:
#Definiendo varias metricas para link prediction

def commonNeighbors(G,source,target):
  adj = dict(G.adj[source])
  adj2 = dict(G.adj[target])
  commonNeighbors = set(adj.keys()) & set(adj2.keys())
  return commonNeighbors

def jaccardCoeff(G,source,target):
  adj = dict(G.adj[source])
  adj2 = dict(G.adj[target])
  unionNeighbors = set(adj.keys()) | set(adj2.keys())
  commonNeighbors = set(adj.keys()) & set(adj2.keys())
  return len(commonNeighbors)/len(unionNeighbors)

def adamicAdar(G,source,target):
  cn = commonNeighbors(G,source,target)
  coeff = 0
  for neighbor in cn:
    coeff += 1/np.log(len(dict(G.adj[neighbor]).keys()))
  return coeff


In [198]:
AdamicAdar(G_data,"Vaca, C.","Ochoa, X.")

2.2172545575960854

In [222]:
#Obteniendo las features, y armando los arrays para usarlo en la regresion logistica

Xtrain =[]
Ytrain =[]
for i in data.values:
  Xtrain.append([adamicAdar(G_data,i[0],i[1])])
  Ytrain.append([i[3]])

Xtest = []
Ytest = []
for i in test.values:
  Xtest.append([adamicAdar(G_data,i[0],i[1])])
  Ytest.append([i[3]])

#Formateando los datos
Xtrain = np.array(Xtrain)
Xtrain.reshape(-1,1)
Xtest = np.array(Xtest)
Xtest.reshape(-1,1)
Ytrain = np.array(Ytrain)
Ytrain.reshape(1,-1)
Ytest = np.array(Ytest)
Ytrain.reshape(1,-1)


array([[0, 0, 0, ..., 1, 1, 1]])

In [223]:
lr = LogisticRegression(class_weight="balanced")

lr.fit(Xtrain, Ytrain)

predictions = lr.predict(Xtest)
print("Regresion logistica",roc_auc_score(Ytest, predictions))

Regresion logistica 0.8867899900981188


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [227]:
# Accuracy de varios tipos de aciertos 

aciertosTotales = 0
unosAcertados = 0
cerosAcertados = 0
suma =0
for tst, y in zip(Xtest,Ytest):
    
    suma+=y[0]
    
    predic = 0
    if lr.predict([tst])[0] > 0.5:
        predic = 1
              
    if y[0] == predic:
        aciertosTotales+=1
        
    if y[0] == 1 and predic == 1:
        unosAcertados+=1
      
    if y[0] == 0 and predic ==0:
        cerosAcertados+=1

acc = aciertosTotales/len(Ytest[:,0])
print("Aciertos totales: ",acc)

acc2 = unosAcertados/suma
print("unos acertados: ",acc2)

acc3 = cerosAcertados/(len(Ytest[:,0])-suma)
print("ceros acertados",acc3)


Aciertos totales:  0.9210992907801419
unos acertados:  0.8476190476190476
ceros acertados 0.9259609325771897
